### Overview of the SIR model

#### Add comments

#### Add diagrams

#### Add assertion for csv - columns, datatypes?

In [1]:
# We create feather file format to investigate if it speeds up data loading
import numpy as np
import pandas as pd
import feather
path = 'data/nyc_od.csv'
OD = pd.read_csv(path, header=None, dtype=np.float64)
path = 'data/nyc_od.feather'
feather.write_dataframe(OD, path)

## Baseline Code

In [2]:
%load_ext line_profiler

In [3]:
import numpy as np
from tqdm import tqdm
import argparse
from multiprocessing import cpu_count
from functools import partial
from multiprocessing.pool import Pool
import pandas as pd

# Matplotlib doesn't play nicely with multiprocessing, so
# we have to create a separate graphing function & import matplotlib inside of that.
def graphing_function(args, days, norms):
    import matplotlib.pyplot as plt
    susceptible_pop_norm, infected_pop_norm, recovered_pop_norm = norms
    public_trans_vec = np.asarray([float(i.replace('[', '').replace(',','').replace(']','')) for i in args.public_trans_vec])
    # plot results and save the plot
    fig = plt.figure()
    ax = plt.axes()
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.plot(np.arange(days), susceptible_pop_norm, label='Susceptible', color='#4aa5f0', linewidth=2)
    ax.plot(np.arange(days), infected_pop_norm, label='Infected', color='#f03737', linewidth=2)
    ax.plot(np.arange(days), recovered_pop_norm, label='Recovered', color='#82e88a', linewidth=2)
    ax.plot(np.arange(days), public_trans_vec, '--', label='Transit', color='k', linewidth=2)
    ax.legend(frameon=False)
    ax.set_xlabel("Days")
    ax.set_ylabel("Share of Population")
    if 'fig_name' in args:
        ax.figure.savefig('figures/' + args.fig_name + '.png')
    else:
        ax.figure.savefig('figures/sir_plot.png')
    plt.close()
    
# Convenience function: access Python dict keys as dict.key instead of dict['key']
class objdict(dict):
    def __getattr__(self, name):
        if name in self:
            return self[name]
        else:
            raise AttributeError("No such attribute: " + name)

    def __setattr__(self, name, value):
        self[name] = value

    def __delattr__(self, name):
        if name in self:
            del self[name]
        else:
            raise AttributeError("No such attribute: " + name)
            
# Run n simulations in parallel: for each run, parameters are randomly sampled from an interval
# NOTE: Are these intervals reasonable?
def multiprocess(args):

    chunks = []
    num_procs = args.randomize
    for _ in range(num_procs):
        d = dict()
        d['origin_matrix_path'] = args.origin_matrix_path
        d['thresh1'] = np.random.randint(0, 350)
        d['thresh2'] = np.random.randint(d['thresh1'], 700)
        d['infection_magnitude'] = np.random.randint(5, 15)
        d['beta'] = args.beta
        d['gamma'] = args.gamma
        d['public_trans'] = np.random.randint(1, 7) * 0.1
        d['public_trans_vec'] = np.asarray([i.replace('[', '').replace(',','').replace(']','') for i in args.public_trans_vec if i!=']'])
        d['days'] = args.days
        d['fig_name'] = 'public_trans_{:0.1f}_({}, {})_inf_{}'.format(d['public_trans'], d['thresh1'], d['thresh2'], d['infection_magnitude'])
        d['loading'] = args.loading
        d = objdict(d)
        chunks.append(d)

    with Pool(num_procs) as p:
        p.map(covid_sim, chunks)

    p.join()

def covid_sim(args):
    # Read in origin-destination flow matrix
    if int(args.loading) == 0:
        OD = np.genfromtxt(args.origin_matrix_path, delimiter=',')
        
    elif int(args.loading) == 1:
        OD = pd.read_csv('data/nyc_od.csv', header=None, dtype=np.float64).to_numpy()
        
    elif int(args.loading) == 2:
        OD = feather.read_dataframe('data/nyc_od.feather').to_numpy()
    

    # initialize the population vector from the origin-destination flow matrix
    N_k = np.abs(np.diagonal(OD) + OD.sum(axis=0) - OD.sum(axis=1))
    locs_len = len(N_k)                 # number of locations
    SIR = np.zeros(shape=(locs_len, 3)) # make a numpy array with 3 columns for keeping track of the S, I, R groups
    SIR[:,0] = N_k                      # initialize the S group with the respective populations
    thresh1 = args.thresh1
    thresh2 = args.thresh2

    first_infections = np.where((thresh1 <= SIR[:, 0]) & (SIR[:, 0] <= thresh2), np.random.randint(1, args.infection_magnitude), 0)   # for demo purposes, randomly introduce infections
    # NOTE: this is arbitrary but not actually random.... 
    SIR[:, 0] = SIR[:, 0] - first_infections
    SIR[:, 1] = SIR[:, 1] + first_infections                           # move infections to the I group

    # row normalize the SIR matrix for keeping track of group proportions
    row_sums = SIR.sum(axis=1)
    SIR_n = SIR / row_sums[:, np.newaxis]

    # initialize parameters
    beta = args.beta
    gamma = args.gamma
    public_trans = args.public_trans                                 # alpha
    R0 = beta/gamma
    beta_vec = np.random.gamma(beta, 2, locs_len)
    gamma_vec = np.full(locs_len, gamma)
    public_trans_vec = args.public_trans_vec
    if isinstance(public_trans_vec[0], str):
        public_trans_vec = np.asarray([float(i.replace('[', '').replace(',','').replace(']','')) for i in args.public_trans_vec if i!=']'])

    # make copy of the SIR matrices 
    SIR_sim = SIR.copy()
    SIR_nsim = SIR_n.copy()

    # run model
    #print(SIR_sim.sum(axis=0).sum() == N_k.sum())
    infected_pop_norm = []
    susceptible_pop_norm = []
    recovered_pop_norm = []
    days = args.days
    for time_step in tqdm(range(days)):
        infected_mat = np.array([SIR_nsim[:,1],]*locs_len).transpose()
        OD_infected = np.round(OD*infected_mat)
        inflow_infected = OD_infected.sum(axis=0)
        inflow_infected = np.round(inflow_infected*public_trans_vec[time_step])
        #print('total infected inflow: ', inflow_infected.sum())
        new_infect = beta_vec*SIR_sim[:, 0]*inflow_infected/(N_k + OD.sum(axis=0))
        new_recovered = gamma_vec*SIR_sim[:, 1]
        new_infect = np.where(new_infect>SIR_sim[:, 0], SIR_sim[:, 0], new_infect)
        SIR_sim[:, 0] = SIR_sim[:, 0] - new_infect
        SIR_sim[:, 1] = SIR_sim[:, 1] + new_infect - new_recovered
        SIR_sim[:, 2] = SIR_sim[:, 2] + new_recovered
        SIR_sim = np.where(SIR_sim<0,0,SIR_sim)
        # recompute the normalized SIR matrix
        row_sums = SIR_sim.sum(axis=1)
        SIR_nsim = SIR_sim / row_sums[:, np.newaxis]
        S = SIR_sim[:,0].sum()/N_k.sum()
        I = SIR_sim[:,1].sum()/N_k.sum()
        R = SIR_sim[:,2].sum()/N_k.sum()
        #print(S, I, R, (S+I+R)*N_k.sum(), N_k.sum())
        #print('\n')
        infected_pop_norm.append(I)
        susceptible_pop_norm.append(S)
        recovered_pop_norm.append(R)

    graphing_function(args, days, (susceptible_pop_norm, infected_pop_norm, recovered_pop_norm))

def covid_sim_serial(args):
    for i in range(args.randomize):
        covid_sim(args)

In [4]:
parser = argparse.ArgumentParser(description='COVID Simulator')
parser.add_argument('--randomize', type=int, default=0)
parser.add_argument('--origin-matrix-path', default='data/nyc_od.csv')
parser.add_argument('--thresh1', type=int, default=0)
parser.add_argument('--thresh2', type=int, default=100)
parser.add_argument('--infection-magnitude', type=int, default=10)
parser.add_argument('--beta', type=float, default=0.75)
parser.add_argument('--gamma', type=float, default=0.2)
parser.add_argument('--public-trans', type=float, default=0.5)
parser.add_argument('--public-trans-vec', nargs='+', default=[])
parser.add_argument('--days', type=int, default=100)
parser.add_argument('--img-folder', default='figures/')
parser.add_argument('--loading', default = 0)
parser.add_argument('--fig_name', default = 'sir_plot')

_StoreAction(option_strings=['--fig_name'], dest='fig_name', nargs=None, const=None, default='sir_plot', type=None, choices=None, help=None, metavar=None)

In [5]:
ptv = np.full(100, 0.56)

In [6]:
args1 = parser.parse_args(f'--randomize 4 --gamma 0.2 --days 100 --loading 0 --public-trans-vec {ptv}'.split())

### Line Profiler Analysis

Using lprun, we see that most of the time is spent loading in the Origin-Destination flow matrix (accounts for 39.2% of the time) and computing the element-wise matrix multiplication OD\*infected_mat and rounding it (accounts for 43.8% of the time). Note that the matrix multiplication occurs in a for-loop that runs multiple times while the loading of the matrix occurs only once. The percentage of time needed to perform all of the matrix multiplications at all of the time steps is roughly the same as the time needed to load in the matrix. We will first focus on trying read the OD matrix faster.

In [7]:
%lprun -f covid_sim covid_sim(args1)

100%|██████████| 100/100 [00:06<00:00, 16.65it/s]


In [8]:
%lprun -f covid_sim_serial covid_sim_serial(args1)

100%|██████████| 100/100 [00:05<00:00, 17.54it/s]


In [7]:
%lprun -f multiprocess multiprocess(args1)

100%|██████████| 100/100 [00:15<00:00,  7.41it/s]


We try data loading using pandas, given by loading argument value of 1 and feather, given by loading argument value of 2.

In [9]:
args2 = parser.parse_args(f'--randomize 4 --gamma 0.2 --days 100 --public-trans-vec {ptv} --loading 1'.split())

In [14]:
args3 = parser.parse_args(f'--randomize 4 --gamma 0.2 --days 100 --public-trans-vec {ptv} --loading 2'.split())

In [30]:
%lprun -f covid_sim_serial covid_sim_serial(args2)

100%|██████████| 100/100 [00:02<00:00, 34.59it/s]


In [31]:
%lprun -f multiprocess multiprocess(args2)

100%|██████████| 100/100 [00:08<00:00, 11.54it/s]


In [11]:
%lprun -f covid_sim_serial covid_sim_serial(args3)

100%|██████████| 100/100 [00:02<00:00, 37.39it/s]


In [12]:
%lprun -f multiprocess multiprocess(args3)

100%|██████████| 100/100 [00:09<00:00, 10.76it/s]

100%|██████████| 100/100 [00:09<00:00, 10.56it/s]


We find that loading data with feather works the fastest, so we'll be using that going forward.

### Improving Matrix Multiplication using Cython

We want to investigate if using Cython can provide a greater speedup.

In [8]:
%load_ext Cython

In [32]:
%%cython

# cython: profile=True
# cython: linetrace=True
# cython: binding=True
# distutils: define_macros=CYTHON_TRACE_NOGIL=1

import numpy as np
from tqdm import tqdm
import argparse
from multiprocessing import cpu_count
from functools import partial
from multiprocessing.pool import Pool
import pandas as pd
import feather

# Matplotlib doesn't play nicely with multiprocessing, so
# we have to create a separate graphing function & import matplotlib inside of that.
def graphing_function(args, days, norms):
    import matplotlib.pyplot as plt
    susceptible_pop_norm, infected_pop_norm, recovered_pop_norm = norms
    public_trans_vec = np.asarray([float(i.replace('[', '').replace(',','').replace(']','')) for i in args.public_trans_vec])
    # plot results and save the plot
    fig = plt.figure()
    ax = plt.axes()
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.plot(np.arange(days), susceptible_pop_norm, label='Susceptible', color='#4aa5f0', linewidth=2)
    ax.plot(np.arange(days), infected_pop_norm, label='Infected', color='#f03737', linewidth=2)
    ax.plot(np.arange(days), recovered_pop_norm, label='Recovered', color='#82e88a', linewidth=2)
    ax.plot(np.arange(days), public_trans_vec, '--', label='Transit', color='k', linewidth=2)
    ax.legend(frameon=False)
    ax.set_xlabel("Days")
    ax.set_ylabel("Share of Population")
    if 'fig_name' in args:
        ax.figure.savefig('figures/' + args.fig_name + '.png')
    else:
        ax.figure.savefig('figures/sir_plot.png')
    plt.close()
    
# Convenience function: access Python dict keys as dict.key instead of dict['key']
class objdict(dict):
    def __getattr__(self, name):
        if name in self:
            return self[name]
        else:
            raise AttributeError("No such attribute: " + name)

    def __setattr__(self, name, value):
        self[name] = value

    def __delattr__(self, name):
        if name in self:
            del self[name]
        else:
            raise AttributeError("No such attribute: " + name)
            
# Run n simulations in parallel: for each run, parameters are randomly sampled from an interval
# NOTE: Are these intervals reasonable?
def multiprocess(args):

    chunks = []
    num_procs = args.randomize
    for _ in range(num_procs):
        d = dict()
        d['origin_matrix_path'] = args.origin_matrix_path
        d['thresh1'] = np.random.randint(0, 350)
        d['thresh2'] = np.random.randint(d['thresh1'], 700)
        d['infection_magnitude'] = np.random.randint(5, 15)
        d['beta'] = args.beta
        d['gamma'] = args.gamma
        d['public_trans'] = np.random.randint(1, 7) * 0.1
        d['public_trans_vec'] = np.asarray([i.replace('[', '').replace(',','').replace(']','') for i in args.public_trans_vec if i!=']'])
        d['days'] = args.days
        d['fig_name'] = 'public_trans_{:0.1f}_({}, {})_inf_{}'.format(d['public_trans'], d['thresh1'], d['thresh2'], d['infection_magnitude'])
        d['loading'] = args.loading
        d = objdict(d)
        chunks.append(d)

    with Pool(num_procs) as p:
        p.map(covid_sim, chunks)

    p.join()

def covid_sim(args):
    # Read in origin-destination flow matrix
    if int(args.loading) == 0:
        OD = np.genfromtxt(args.origin_matrix_path, delimiter=',')
        
    elif int(args.loading) == 1:
        OD = pd.read_csv('data/nyc_od.csv', header=None, dtype=np.float64).to_numpy()
        
    elif int(args.loading) == 2:
        OD = feather.read_dataframe('data/nyc_od.feather').to_numpy()
    

    # initialize the population vector from the origin-destination flow matrix
    N_k = np.abs(np.diagonal(OD) + OD.sum(axis=0) - OD.sum(axis=1))
    locs_len = len(N_k)                 # number of locations
    SIR = np.zeros(shape=(locs_len, 3)) # make a numpy array with 3 columns for keeping track of the S, I, R groups
    SIR[:,0] = N_k                      # initialize the S group with the respective populations
    thresh1 = args.thresh1
    thresh2 = args.thresh2

    first_infections = np.where((thresh1 <= SIR[:, 0]) & (SIR[:, 0] <= thresh2), np.random.randint(1, args.infection_magnitude), 0)   # for demo purposes, randomly introduce infections
    # NOTE: this is arbitrary but not actually random.... 
    SIR[:, 0] = SIR[:, 0] - first_infections
    SIR[:, 1] = SIR[:, 1] + first_infections                           # move infections to the I group

    # row normalize the SIR matrix for keeping track of group proportions
    row_sums = SIR.sum(axis=1)
    SIR_n = SIR / row_sums[:, np.newaxis]

    # initialize parameters
    beta = args.beta
    gamma = args.gamma
    public_trans = args.public_trans                                 # alpha
    R0 = beta/gamma
    beta_vec = np.random.gamma(beta, 2, locs_len)
    gamma_vec = np.full(locs_len, gamma)
    public_trans_vec = args.public_trans_vec
    if isinstance(public_trans_vec[0], str):
        public_trans_vec = np.asarray([float(i.replace('[', '').replace(',','').replace(']','')) for i in args.public_trans_vec if i!=']'])

    # make copy of the SIR matrices 
    SIR_sim = SIR.copy()
    SIR_nsim = SIR_n.copy()

    # run model
    #print(SIR_sim.sum(axis=0).sum() == N_k.sum())
    infected_pop_norm = []
    susceptible_pop_norm = []
    recovered_pop_norm = []
    days = args.days
    for time_step in tqdm(range(days)):
        infected_mat = np.array([SIR_nsim[:,1],]*locs_len).transpose()
        OD_infected = np.round(OD*infected_mat)
        inflow_infected = OD_infected.sum(axis=0)
        inflow_infected = np.round(inflow_infected*public_trans_vec[time_step])
        #print('total infected inflow: ', inflow_infected.sum())
        new_infect = beta_vec*SIR_sim[:, 0]*inflow_infected/(N_k + OD.sum(axis=0))
        new_recovered = gamma_vec*SIR_sim[:, 1]
        new_infect = np.where(new_infect>SIR_sim[:, 0], SIR_sim[:, 0], new_infect)
        SIR_sim[:, 0] = SIR_sim[:, 0] - new_infect
        SIR_sim[:, 1] = SIR_sim[:, 1] + new_infect - new_recovered
        SIR_sim[:, 2] = SIR_sim[:, 2] + new_recovered
        SIR_sim = np.where(SIR_sim<0,0,SIR_sim)
        # recompute the normalized SIR matrix
        row_sums = SIR_sim.sum(axis=1)
        SIR_nsim = SIR_sim / row_sums[:, np.newaxis]
        S = SIR_sim[:,0].sum()/N_k.sum()
        I = SIR_sim[:,1].sum()/N_k.sum()
        R = SIR_sim[:,2].sum()/N_k.sum()
        #print(S, I, R, (S+I+R)*N_k.sum(), N_k.sum())
        #print('\n')
        infected_pop_norm.append(I)
        susceptible_pop_norm.append(S)
        recovered_pop_norm.append(R)

    graphing_function(args, days, (susceptible_pop_norm, infected_pop_norm, recovered_pop_norm))

def covid_sim_serial(args):
    for i in range(args.randomize):
        covid_sim(args)

In [33]:
%lprun -f covid_sim_serial covid_sim_serial(args3)

100%|██████████| 100/100 [00:02<00:00, 34.57it/s]


In [34]:
%lprun -f multiprocess multiprocess(args3)

100%|██████████| 100/100 [00:09<00:00, 10.42it/s]


We see a slight speed up when compiling with Cython without specifying the types of the variables for both serial and multiprocessing.

### Cython + Removing tqdm

We wanted to see if removing tqdm could provide an additional speedup.

In [12]:
%%cython

# cython: profile=True
# cython: linetrace=True
# cython: binding=True
# distutils: define_macros=CYTHON_TRACE_NOGIL=1

import numpy as np
from tqdm import tqdm
import argparse
from multiprocessing import cpu_count
from functools import partial
from multiprocessing.pool import Pool
import pandas as pd
import feather

# Matplotlib doesn't play nicely with multiprocessing, so
# we have to create a separate graphing function & import matplotlib inside of that.
def graphing_function(args, days, norms):
    import matplotlib.pyplot as plt
    susceptible_pop_norm, infected_pop_norm, recovered_pop_norm = norms
    public_trans_vec = np.asarray([float(i.replace('[', '').replace(',','').replace(']','')) for i in args.public_trans_vec])
    # plot results and save the plot
    fig = plt.figure()
    ax = plt.axes()
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.plot(np.arange(days), susceptible_pop_norm, label='Susceptible', color='#4aa5f0', linewidth=2)
    ax.plot(np.arange(days), infected_pop_norm, label='Infected', color='#f03737', linewidth=2)
    ax.plot(np.arange(days), recovered_pop_norm, label='Recovered', color='#82e88a', linewidth=2)
    ax.plot(np.arange(days), public_trans_vec, '--', label='Transit', color='k', linewidth=2)
    ax.legend(frameon=False)
    ax.set_xlabel("Days")
    ax.set_ylabel("Share of Population")
    if 'fig_name' in args:
        ax.figure.savefig('figures/' + args.fig_name + '.png')
    else:
        ax.figure.savefig('figures/sir_plot.png')
    plt.close()
    
# Convenience function: access Python dict keys as dict.key instead of dict['key']
class objdict(dict):
    def __getattr__(self, name):
        if name in self:
            return self[name]
        else:
            raise AttributeError("No such attribute: " + name)

    def __setattr__(self, name, value):
        self[name] = value

    def __delattr__(self, name):
        if name in self:
            del self[name]
        else:
            raise AttributeError("No such attribute: " + name)
            
# Run n simulations in parallel: for each run, parameters are randomly sampled from an interval
# NOTE: Are these intervals reasonable?
def multiprocess(args):

    chunks = []
    num_procs = args.randomize
    for _ in range(num_procs):
        d = dict()
        d['origin_matrix_path'] = args.origin_matrix_path
        d['thresh1'] = np.random.randint(0, 350)
        d['thresh2'] = np.random.randint(d['thresh1'], 700)
        d['infection_magnitude'] = np.random.randint(5, 15)
        d['beta'] = args.beta
        d['gamma'] = args.gamma
        d['public_trans'] = np.random.randint(1, 7) * 0.1
        d['public_trans_vec'] = np.asarray([i.replace('[', '').replace(',','').replace(']','') for i in args.public_trans_vec])
        d['days'] = args.days
        d['fig_name'] = 'public_trans_{:0.1f}_({}, {})_inf_{}'.format(d['public_trans'], d['thresh1'], d['thresh2'], d['infection_magnitude'])
        d['loading'] = args.loading
        d = objdict(d)
        chunks.append(d)

    with Pool(num_procs) as p:
        p.map(covid_sim, chunks)

    p.join()

def covid_sim(args):
    # Read in origin-destination flow matrix
    if int(args.loading) == 0:
        OD = np.genfromtxt(args.origin_matrix_path, delimiter=',')
        
    elif int(args.loading) == 1:
        OD = pd.read_csv('data/nyc_od.csv', header=None, dtype=np.float64).to_numpy()
        
    elif int(args.loading) == 2:
        OD = feather.read_dataframe('data/nyc_od.feather').to_numpy()
    

    # initialize the population vector from the origin-destination flow matrix
    N_k = np.abs(np.diagonal(OD) + OD.sum(axis=0) - OD.sum(axis=1))
    locs_len = len(N_k)                 # number of locations
    SIR = np.zeros(shape=(locs_len, 3)) # make a numpy array with 3 columns for keeping track of the S, I, R groups
    SIR[:,0] = N_k                      # initialize the S group with the respective populations
    thresh1 = args.thresh1
    thresh2 = args.thresh2

    first_infections = np.where((thresh1 <= SIR[:, 0]) & (SIR[:, 0] <= thresh2), np.random.randint(1, args.infection_magnitude), 0)   # for demo purposes, randomly introduce infections
    # NOTE: this is arbitrary but not actually random.... 
    SIR[:, 0] = SIR[:, 0] - first_infections
    SIR[:, 1] = SIR[:, 1] + first_infections                           # move infections to the I group

    # row normalize the SIR matrix for keeping track of group proportions
    row_sums = SIR.sum(axis=1)
    SIR_n = SIR / row_sums[:, np.newaxis]

    # initialize parameters
    beta = args.beta
    gamma = args.gamma
    public_trans = args.public_trans                                 # alpha
    R0 = beta/gamma
    beta_vec = np.random.gamma(beta, 2, locs_len)
    gamma_vec = np.full(locs_len, gamma)
    public_trans_vec = args.public_trans_vec
    if isinstance(public_trans_vec[0], str):
        public_trans_vec = np.asarray([float(i.replace('[', '').replace(',','').replace(']','')) for i in public_trans_vec if i != ']'])

    # make copy of the SIR matrices 
    SIR_sim = SIR.copy()
    SIR_nsim = SIR_n.copy()

    # run model
    #print(SIR_sim.sum(axis=0).sum() == N_k.sum())
    infected_pop_norm = []
    susceptible_pop_norm = []
    recovered_pop_norm = []
    days = args.days
    for time_step in range(days):
        infected_mat = np.array([SIR_nsim[:,1],]*locs_len).transpose()
        OD_infected = np.round(OD*infected_mat)
        inflow_infected = OD_infected.sum(axis=0)
        inflow_infected = np.round(inflow_infected*public_trans_vec[time_step])
        #print('total infected inflow: ', inflow_infected.sum())
        new_infect = beta_vec*SIR_sim[:, 0]*inflow_infected/(N_k + OD.sum(axis=0))
        new_recovered = gamma_vec*SIR_sim[:, 1]
        new_infect = np.where(new_infect>SIR_sim[:, 0], SIR_sim[:, 0], new_infect)
        SIR_sim[:, 0] = SIR_sim[:, 0] - new_infect
        SIR_sim[:, 1] = SIR_sim[:, 1] + new_infect - new_recovered
        SIR_sim[:, 2] = SIR_sim[:, 2] + new_recovered
        SIR_sim = np.where(SIR_sim<0,0,SIR_sim)
        # recompute the normalized SIR matrix
        row_sums = SIR_sim.sum(axis=1)
        SIR_nsim = SIR_sim / row_sums[:, np.newaxis]
        S = SIR_sim[:,0].sum()/N_k.sum()
        I = SIR_sim[:,1].sum()/N_k.sum()
        R = SIR_sim[:,2].sum()/N_k.sum()
        #print(S, I, R, (S+I+R)*N_k.sum(), N_k.sum())
        #print('\n')
        infected_pop_norm.append(I)
        susceptible_pop_norm.append(S)
        recovered_pop_norm.append(R)

    graphing_function(args, days, (susceptible_pop_norm, infected_pop_norm, recovered_pop_norm))

def covid_sim_serial(args):
    for i in range(args.randomize):
        covid_sim(args)

In [15]:
# fastest process - checking overall improvement for 1 run
%lprun -f covid_sim covid_sim(args3)

In [36]:
%lprun -f covid_sim_serial covid_sim_serial(args3)

In [37]:
%lprun -f multiprocess multiprocess(args3)

Using feature for loading, using Cython and removing tqdm provided a time reduction for both serial and parallel processes.

### Cython - with types specified and multiprocessing

In [9]:
%load_ext Cython
%matplotlib inline

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [10]:
%%cython

# cython: profile=True
# cython: linetrace=True
# cython: binding=True
# distutils: define_macros=CYTHON_TRACE_NOGIL=1

import numpy as np
from tqdm import tqdm
import argparse
from multiprocessing import cpu_count
from functools import partial
from multiprocessing.pool import Pool
import pandas as pd
cimport numpy as np

def graphing_function(np.ndarray[np.float64_t, ndim=1] infected_pop_norm, 
                      np.ndarray[np.float64_t, ndim=1] susceptible_pop_norm, 
                      np.ndarray[np.float64_t, ndim=1] recovered_pop_norm, 
                      np.ndarray[np.float64_t, ndim=1] public_trans_vec, 
                      int days, 
                      fig_name):
    import matplotlib.pyplot as plt
    # plot results and save the plot
    fig = plt.figure()
    ax = plt.axes()
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.plot(np.arange(days), susceptible_pop_norm, label='Susceptible', color='#4aa5f0', linewidth=2)
    ax.plot(np.arange(days), infected_pop_norm, label='Infected', color='#f03737', linewidth=2)
    ax.plot(np.arange(days), recovered_pop_norm, label='Recovered', color='#82e88a', linewidth=2)
    ax.plot(np.arange(days), public_trans_vec, '--', label='Transit', color='k', linewidth=2)
    ax.legend(frameon=False, loc='center left', bbox_to_anchor=(1.01, 0.5))
    ax.set_xlabel("Days")
    ax.set_ylabel("Share of Population")
    if fig_name:
        ax.figure.savefig('figures/' + fig_name + '.png')
    else:
        ax.figure.savefig('figures/sir_plot.png')
    # plt.show()
    plt.close()
    
# Convenience function: access Python dict keys as dict.key instead of dict['key']
class objdict(dict):
    def __getattr__(self, name):
        if name in self:
            return self[name]
        else:
            raise AttributeError("No such attribute: " + name)

    def __setattr__(self, name, value):
        self[name] = value

    def __delattr__(self, name):
        if name in self:
            del self[name]
        else:
            raise AttributeError("No such attribute: " + name)
            
# Run n simulations in parallel: for each run, parameters are randomly sampled from an interval
# NOTE: Are these intervals reasonable?
def multiprocess(args):

    chunks = []
    num_procs = args.randomize
    for _ in range(num_procs):
        d = dict()
        d['origin_matrix_path'] = args.origin_matrix_path
        d['thresh1'] = np.random.randint(0, 350)
        d['thresh2'] = np.random.randint(d['thresh1'], 700)
        d['infection_magnitude'] = np.random.randint(5, 15)
        d['beta'] = args.beta
        d['gamma'] = args.gamma
        d['public_trans'] = np.random.randint(1, 7) * 0.1
        d['public_trans_vec'] = np.asarray([float(i.replace('[', '').replace(',','').replace(']','')) for i in args.public_trans_vec])
        d['days'] = args.days
        d['fig_name'] = 'public_trans_{:0.1f}_({}, {})_inf_{}'.format(d['public_trans'], d['thresh1'], d['thresh2'], d['infection_magnitude'])
        d['loading'] = args.loading
        d = objdict(d)
        chunks.append(d)

    with Pool(num_procs) as p:
        p.map(covid_sim, chunks)

    p.join()

def covid_sim(args):
    # Read in origin-destination flow matrix  
    cdef np.ndarray[np.float64_t, ndim=2] OD, SIR, SIR_n, SIR_sim, SIR_nsim, infected_mat, OD_infected 
    cdef np.ndarray[np.float64_t, ndim=1] N_k, first_infections, row_sums, beta_vec, gamma_vec, inflow_infected, new_infect, new_recovered #, public_trans_vec
    cdef int locs_len, time_step
    cdef np.float64_t R0, S, I, R
    cdef np.ndarray[np.float64_t, ndim=1] infected_pop_norm, susceptible_pop_norm, recovered_pop_norm 

    if args.loading != 0:
        OD = pd.read_csv('data/nyc_od.csv', header=None, dtype=np.float64).to_numpy()
    else:
        OD = np.genfromtxt(args.origin_matrix_path, delimiter=',')

    # initialize the population vector from the origin-destination flow matrix
    N_k = np.abs(np.diagonal(OD) + OD.sum(axis=0) - OD.sum(axis=1))
    locs_len = len(N_k)                 # number of locations
    SIR = np.zeros(shape=(locs_len, 3)) # make a numpy array with 3 columns for keeping track of the S, I, R groups
    SIR[:,0] = N_k                      # initialize the S group with the respective populations
    thresh1 = args.thresh1
    thresh2 = args.thresh2
    first_infections = np.where((thresh1 <= SIR[:, 0]) & (SIR[:, 0] <= thresh2), np.random.randint(1, args.infection_magnitude), 0).astype(np.float64)   # for demo purposes, randomly introduce infections
    # NOTE: this is arbitrary but not actually random.... 
    SIR[:, 0] = SIR[:, 0] - first_infections
    SIR[:, 1] = SIR[:, 1] + first_infections                           # move infections to the I group

    # row normalize the SIR matrix for keeping track of group proportions
    row_sums = SIR.sum(axis=1)
    SIR_n = SIR / row_sums[:, np.newaxis]

    # initialize parameters
    beta = args.beta
    gamma = args.gamma
    public_trans = args.public_trans                                 # alpha
    R0 = beta/gamma
    beta_vec = np.random.gamma(beta, 2, locs_len)
    gamma_vec = np.full(locs_len, gamma)
    public_trans_vec = args.public_trans_vec
    if isinstance(public_trans_vec[0], str):
        public_trans_vec = np.asarray([float(i.replace('[', '').replace(',','').replace(']','')) for i in public_trans_vec])
        
    fig_name = args.fig_name

    # make copy of the SIR matrices 
    SIR_sim = SIR.copy()
    SIR_nsim = SIR_n.copy()

    # run model
    #print(SIR_sim.sum(axis=0).sum() == N_k.sum())
    days = args.days
    infected_pop_norm = np.zeros(days)
    susceptible_pop_norm = np.zeros(days)
    recovered_pop_norm = np.zeros(days)
    for time_step in tqdm(range(days)):
        infected_mat = np.array([SIR_nsim[:,1],]*locs_len).transpose()
        OD_infected = np.round(OD*infected_mat)
        inflow_infected = OD_infected.sum(axis=0)
        inflow_infected = np.round(inflow_infected*public_trans_vec[time_step])
        #print('total infected inflow: ', inflow_infected.sum())
        new_infect = beta_vec*SIR_sim[:, 0]*inflow_infected/(N_k + OD.sum(axis=0))
        new_recovered = gamma_vec*SIR_sim[:, 1]
        new_infect = np.where(new_infect>SIR_sim[:, 0], SIR_sim[:, 0], new_infect)
        SIR_sim[:, 0] = SIR_sim[:, 0] - new_infect
        SIR_sim[:, 1] = SIR_sim[:, 1] + new_infect - new_recovered
        SIR_sim[:, 2] = SIR_sim[:, 2] + new_recovered
        SIR_sim = np.where(SIR_sim<0,0,SIR_sim)
        # recompute the normalized SIR matrix
        row_sums = SIR_sim.sum(axis=1)
        SIR_nsim = SIR_sim / row_sums[:, np.newaxis]
        S = SIR_sim[:,0].sum()/N_k.sum()
        I = SIR_sim[:,1].sum()/N_k.sum()
        R = SIR_sim[:,2].sum()/N_k.sum()
        #print(S, I, R, (S+I+R)*N_k.sum(), N_k.sum())
        #print('\n')
        infected_pop_norm[time_step] = I 
        susceptible_pop_norm[time_step] = S 
        recovered_pop_norm[time_step] = R

    graphing_function(infected_pop_norm, susceptible_pop_norm, recovered_pop_norm, public_trans_vec, days, fig_name)
    
def covid_sim_serial(args):
    for i in range(args.randomize):
        covid_sim(args)

In [42]:
%lprun -f covid_sim_serial covid_sim_serial(args3)

100%|██████████| 100/100 [00:02<00:00, 34.74it/s]


In [43]:
%lprun -f multiprocess multiprocess(args3)

100%|██████████| 100/100 [00:09<00:00, 10.21it/s]


